In [1]:
from gigachat_api import GigaChat

gigachat = GigaChat()

In [6]:
import requests
requests.packages.urllib3.disable_warnings() 
import pandas as pd

filename = 'data/data_new_2.xlsx'
df = gigachat.process(filename)
df.head()

124\123

,chunk,keywords
0,Всем добрый день ! Это курс ведения в програм...,1. Курс ведения в программирование\n2. Обзорна...
1,"Данный курс , вот этот курс именно введения в...",1. Курс по введению в программирование на осно...
2,"Давайте посмотрим , что же я все-таки вам при...",1. Научные факультеты\n2. Эксперименты\n3. Дан...
3,"То есть , чтобы именно можно было принимать п...",1. Цифровое поле\n2. Законы\n3. Медовселенная\...
4,"А вот то , что вы написали про распознавание ...",1. Распознавание речи\n2. Искусственный интелл...


In [3]:
import re
import itertools
data = []


In [7]:

for index, row in df.iterrows():
    try:
        keywords = re.split(r'\n\d+\.', row.keywords[3:].lower())
        new_keywords = []
        for key in keywords:
            tmp = re.split(r'\s+', re.sub(r'^\s+|\s+$', '', key))
            if 6 > len(tmp) > 1:
                perms = list(itertools.permutations(tmp))
                perms = [' '.join(items) for items in perms]
                new_keywords.extend(perms)
            else:
                new_keywords.append(tmp[0])

        data.append([re.split(r'\s+', row.chunk.lower()), new_keywords])
    except:
        raise
        continue

In [8]:
len(data)

402

In [9]:
pd.DataFrame(data=data, columns=['chunk', 'keywords']).to_excel('data_with_keywords_2.xlsx', index=False)

In [10]:
import ast
df = pd.read_excel('data_with_keywords_2.xlsx')

chunks = []
markups = []
for index, row in df.iterrows():
    chunk = ast.literal_eval(row.chunk)
    # print(chunk)

    bio_markup = []
    i = -1
    while i < len(chunk) - 1:
        i += 1
        if re.search(r'[A-Za-zА-ЯЁа-яё]', chunk[i]) is None:
            bio_markup.append('O')
            continue
        was_added = False
        for key in ast.literal_eval(row.keywords):

            # print(key)
            # print(key, '|', chunk[i])
            parts = re.split(r'\s+', key)

            if len(parts) == 1:
                if chunk[i] == key:
                    bio_markup.append('B')
                    was_added=True
                    break
                else:
                    continue
                    # bio_markup.append('O')
            else:
                # print(len(chunk), i, len(parts))
                if i + len(parts) < len(chunk):
                    cur = []
                    if chunk[i] == parts[0]:
                        cur.append('B')
                    else:
                        continue
                    # print(parts, chunk)
                    for j in range(1, len(parts)):
                        # print(parts)
                        # print(j)
                        # print(parts[j])
                        if chunk[i+j] == parts[j]:
                            cur.append('I')
                        else:
                            break
                    # print(cur)
                    if len(cur) == len(parts):
                        i = i + len(parts) - 1
                        was_added=True
                        bio_markup.extend(cur)
                        break
                else:
                    continue
                    # bio_markup.append('O')
        if not was_added:
            bio_markup.append('O')

        # print(len(bio_markup), len(chunk))

    if len(chunk) > len(bio_markup):
        x = len(bio_markup)
        chunk = chunk[:x]
    
    # print(chunk)
    # print(len(chunk), len(bio_markup))

    chunks.append(chunk)
    markups.append(bio_markup)

In [13]:
len(chunks), len(markups)

(402, 402)

In [14]:
corr = {'O': 0, 'B': 1, 'I': 2}

for i in range(len(markups)):
    for j in range(len(markups[i])):
        markups[i][j] = corr[markups[i][j]]

In [17]:
markups

KeyboardInterrupt: 

In [15]:
pd.DataFrame({'tokens': chunks, 'ner_tags': markups}).to_csv('bio_2.csv', index=False)

In [16]:
df_1 = pd.read_csv('data/bio.csv')
df_2 = pd.read_csv('data/bio_2.csv')
df = pd.concat([df_1, df_2])
df.head()

,tokens,ner_tags
0,"['', 'друзья', ',', 'привет', '!', 'с', 'вами'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"['', 'первая', 'характеристика', '–', 'это', '...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,"['', 'далее', ',', 'мы', 'указываем', 'на', 'м...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"['', 'итак', ',', 'укажем', 'тип', 'данных', '...","[0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,"['', 'для', 'вывода', 'элементов', ',', 'для',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, ..."


In [ ]:
df.to_csv('data/bio_concat.csv', index=False)

In [ ]:
re.split(' ', 'privet')